In [1]:
import pandas as pd
import numpy as np
import os

# NNG, NNP 형태소만을 통해 BOW 구성
def read_files(directory_path):
    data = []

    # 디렉토리 내 모든 파일에 대해 반복
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)

        # 파일인지 확인
        if os.path.isfile(file_path):
            base_filename = os.path.basename(filename)  # 파일 이름

            with open(file_path, 'r', encoding='utf-8') as file:
                for line in file:
                    line = line.strip()  # 줄 양 끝의 공백 제거
                    if line:  # 비어 있지 않은 줄만 처리
                        try:
                            parts = line.split()  # 공백으로 분리
                            # print('parts : ',parts)
                            if len(parts) > 1:  # 필요한 부분이 있는지 확인
                              pos = parts[1]
                              word = parts[0]

                              if pos in ['NNG', 'NNP']:
                                      data.append((base_filename, word, pos))
                              # else:
                                  # print(f"Unexpected format in {filename}")

                        except ValueError as e:
                            print(f"Skipping line in {filename}: {line} - Error: {e}")

    # DataFrame 생성
    df = pd.DataFrame(data, columns=["file name", "word", "pos" ])
    return df

business_data = read_files('/content/drive/MyDrive/NLP labs/Week 9_lab 3/Week9_1_IntroToNLP_BusinessReport/business_report_kkma')

# remove duplications
business_data = business_data.drop_duplicates(subset=['file name','word'])
business_data.head()

,file name,word,pos
0,000070.txt,사업,NNG
1,000070.txt,내용,NNG
3,000070.txt,개요,NNG
4,000070.txt,업계,NNG
5,000070.txt,현황,NNG


In [2]:
# 불용어 정의_30개 이상
stopwords = [
    '액','배','및','등','위','것','가','을','를',
    "그", "이것", "저것", "이", "나", "너", "우리", "당신",
    "그들", "것", "에", "에서", "의", "와", "과", "은",
    "는", "이다", "같다", "하다", "모두", "또한", "그런",
    "이렇다", "저렇다", "이렇게", "저렇게", "만약", "그래서",
    "하지만"
]

# CountVectorizer
# !pip install konlpy
# from konlpy.tag import Okt
# okt=Okt()

# def okt_tokenizer(text):
#     return okt.morphs(text) -> RAM 다운 ...

from sklearn.feature_extraction.text import CountVectorizer


vectorizer = CountVectorizer(max_features=1000, stop_words=stopwords)  # maximum_1000

# create BOW
X = vectorizer.fit_transform(business_data['word'])

# result
bag_of_words_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
bag_of_words_df['file name'] = business_data['file name'].str[:-4]
bag_of_words_df = bag_of_words_df.dropna(subset=['file name']) # nan값 제거
print(bag_of_words_df)

        가격  가공  가능  가능성  가동  가동률  가속화  가스  가액  가운데  ...  회수  회피  획득  효과  효과적  \
0        0   0   0    0   0    0    0   0   0    0  ...   0   0   0   0    0   
1        0   0   0    0   0    0    0   0   0    0  ...   0   0   0   0    0   
3        0   0   0    0   0    0    0   0   0    0  ...   0   0   0   0    0   
4        0   0   0    0   0    0    0   0   0    0  ...   0   0   0   0    0   
5        0   0   0    0   0    0    0   0   0    0  ...   0   0   0   0    0   
...     ..  ..  ..  ...  ..  ...  ...  ..  ..  ...  ...  ..  ..  ..  ..  ...   
914301   0   0   0    0   0    0    0   0   0    0  ...   0   0   0   0    0   
914302   0   0   0    0   0    0    0   0   0    0  ...   0   0   0   0    0   
914304   0   0   0    0   0    0    0   0   0    0  ...   0   0   0   0    0   
914306   0   0   0    0   0    0    0   0   0    0  ...   0   0   0   0    0   
914308   0   0   0    0   0    0    0   0   0    0  ...   0   0   0   0    0   

        효율  효율성  효율적  흐름  file name  
0

In [9]:
# convert into TF-IDF matrix through Bag Of Words using TfidfTransformer
from sklearn.feature_extraction.text import TfidfTransformer

# Create TF-IDF Transformer
tfidf_transformer = TfidfTransformer()

# Transform Bag of Words to TF-IDF matrix

X = bag_of_words_df.drop(columns=['file name'])
tfidf_matrix = tfidf_transformer.fit_transform(X.values)

# Convert the TF-IDF matrix to a DataFrame for better readability
TFidf_data = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
print(TFidf_data)


         가격   가공   가능  가능성   가동  가동률  가속화   가스   가액  가운데  ...  회사명   회수   회피  \
0       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
...     ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
187953  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
187954  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
187955  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
187956  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
187957  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

         획득   효과  효과적   효율  효율성  효율적   

In [57]:
# TruncatedSVD, 10 initial topics

def svd_topics(initial_topicNum):
  from sklearn.decomposition import TruncatedSVD
  svd = TruncatedSVD(n_components=initial_topicNum)
  truncated_svd = svd.fit_transform(tfidf_matrix)

  # Convert the SVD result to a DataFrame for better readability
  svd_df = pd.DataFrame(truncated_svd, columns=[f'Topic {i+1}' for i in range(initial_topicNum)])
  print(svd_df) # 여기서 가장 높은 값이 나오는게 가장 연관이 있는 topic
  return svd, svd_df


svd_10, svd_10_df= svd_topics(10)

         Topic 1   Topic 2   Topic 3   Topic 4   Topic 5   Topic 6   Topic 7  \
0       0.083034 -0.058155 -0.053502 -0.220397  0.210094 -0.048446  0.097315   
1      -0.010936 -0.038737 -0.025167 -0.033571 -0.183416 -0.006336  0.081867   
2       0.019144  0.009596 -0.017554 -0.012176  0.030162 -0.033395  0.062683   
3       0.097002  0.055430 -0.128667  0.117570 -0.233041 -0.141844  0.074337   
4       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
...          ...       ...       ...       ...       ...       ...       ...   
187953  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
187954  0.000019  0.000017  0.000073  0.000032  0.000059 -0.000095 -0.000077   
187955  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
187956  0.000106  0.000018  0.000088 -0.000056  0.000103  0.000073  0.000112   
187957  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

         Topic 8   Topic 9  Topic 10  


In [58]:
# print 20 words for each topic
def rep_words(svd, num):
  terms = vectorizer.get_feature_names_out()
  for i in range(svd.components_.shape[0]):
      term_indices = svd.components_[i].argsort()[::-1]  # sort in descending order
      top_terms = [terms[j] for j in term_indices[:num]]  # top 20 words
      print(f"TOPIC {i + 1}: {', '.join(top_terms)}")

rep_words(svd_10, 20)

TOPIC 1: 성장, 상황, 거래, 운영, 판매, 기준, 설비, 현재, 현황, 유지, 매출액, 전략, 수준, 사업, 흐름, 대비, 가격, 상품, 능력, 지속
TOPIC 2: 상품, 비율, 국내, 산업, 매출, 구분, 상황, 기술, 감소, 당사, 기타, 유지, 기초, 최근, 전략, 포함, 재무, 부문, 향후, 한국
TOPIC 3: 국내, 비율, 성장, 주요, 유형, 특성, 설비, 능력, 이익, 실적, 부문, 구성, 기관, 이후, 환율, 천원, 당기, 수준, 영업, 구분
TOPIC 4: 발생, 국내, 유형, 현재, 상황, 자산, 계약, 상품, 파생, 매출, 대상, 현황, 재무, 전략, 설비, 증가, 산업, 관리, 천원, 기관
TOPIC 5: 상품, 사업, 국내, 특성, 거래, 발생, 방법, 추이, 능력, 생산, 연결, 경영, 부문, 실적, 기관, 현재, 개요, 사용, 평균, 가능
TOPIC 6: 증가, 개발, 당사, 단위, 비율, 특성, 개요, 자산, 실적, 상황, 국내, 판매, 참고, 강화, 전략, 유형, 연구, 현금, 대상, 관리
TOPIC 7: 실적, 자산, 확보, 거래, 관리, 변동, 발생, 서비스, 활동, 능력, 개발, 평가, 감소, 사업, 연결, 내용, 제공, 상황, 산업, 현황
TOPIC 8: 운영, 관리, 설비, 특성, 서비스, 확보, 개요, 상승, 전략, 위험, 가능, 담당, 재무, 경로, 경쟁, 유지, 투자, 연구, 생산, 공급
TOPIC 9: 계약, 운영, 개발, 제품, 주요, 국내, 경쟁, 기간, 자산, 해당, 사용, 비용, 형태, 거래, 내용, 수준, 보유, 현황, 기술, 가치
TOPIC 10: 매출, 유지, 개발, 개요, 이익, 부문, 내용, 사업, 당기, 지속적, 환경, 발생, 운영, 금융, 참고, 상승, 가격, 현재, 자본, 파생


In [75]:
# print 10 companies for each topic

# xlsx파일에서 회사 이름 가져오기
companies_data = pd.read_excel('/content/drive/MyDrive/NLP labs/Week 9_lab 3/Week9_1_IntroToNLP_BusinessReport/ticker_name.xlsx')
# companies_data.head()

# # 각 토픽별로 대표 회사 10개 출력 (NaN과 중복 제거)

def print_company(num_topic, num_company):
  svd_n, svd_n_df = svd_topics(num_topic) # 토픽 개수
  n_topics = svd_n_df.shape[1] - 1
  svd_df = svd_n_df

  print(f'\n\n===============================Topic : {num_topic}, company : {num_company}============================================')
  svd_df['file name'] = bag_of_words_df['file name']

  for i in range(num_topic):
      topic_name = f'Topic {i+1}'

      valid_files = svd_df['file name'].dropna().drop_duplicates()
      top_indices = svd_df.loc[valid_files.index][topic_name].nlargest(num_company).index
      top_files = svd_df.loc[top_indices, 'file name'].values


      top_companies = []

      for file_name in top_files:
          # 'file name' 컬럼에서 file_name과 일치하는 행을 찾기
          matched_row = companies_data[companies_data['Ticker'] == int(file_name)]

          # 결과가 있을 경우 리스트에 추가
          if not matched_row.empty:
              top_companies.append(matched_row['Stock Name'].values[0])
          else:
              print(f"No match found for file name: {file_name}")


      print(f"\nMain {num_company} companies for {topic_name}:")
      for file in top_companies:
          print(f"  - {file}")

print_company(10, 10) # 토픽 10개, 회사 10개

         Topic 1   Topic 2   Topic 3   Topic 4   Topic 5   Topic 6   Topic 7  \
0       0.023369  0.056811  0.031956 -0.250193  0.098814 -0.117317 -0.034506   
1      -0.035616  0.033500 -0.049921  0.061246 -0.093622  0.055396 -0.113290   
2      -0.012514 -0.017176  0.015281 -0.019996 -0.016697 -0.028888 -0.002612   
3       0.003956 -0.143503  0.104227 -0.287457  0.118423 -0.183921  0.177336   
4       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
...          ...       ...       ...       ...       ...       ...       ...   
187953  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
187954 -0.000120 -0.000022  0.000092  0.000171 -0.000163 -0.000291 -0.000058   
187955  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
187956 -0.000012  0.000046 -0.000013 -0.000094  0.000139  0.000045  0.000148   
187957  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

         Topic 8   Topic 9  Topic 10  


In [77]:
# 토픽 개수 찾기
print_company(3, 5) # 토픽 3개, 회사 5개
print_company(5, 5) # 토픽 5개, 회사 5개
print_company(6, 5) # 토픽 6개, 회사 5개

         Topic 1   Topic 2   Topic 3
0      -0.083056  0.069599 -0.038593
1      -0.032440 -0.116399  0.111130
2      -0.011850  0.008671  0.016636
3       0.119116  0.043778  0.035652
4       0.000000  0.000000  0.000000
...          ...       ...       ...
187953  0.000000  0.000000  0.000000
187954 -0.000115 -0.000143 -0.000028
187955  0.000000  0.000000  0.000000
187956 -0.000113 -0.000012 -0.000038
187957  0.000000  0.000000  0.000000

[187958 rows x 3 columns]


===============================Topic : 3, company : 5============================================

Main 5 companies for Topic 1:
  - 가온전선
  - 노루홀딩스
  - 경방
  - 동아쏘시오홀딩스
  - 한화손해보험

Main 5 companies for Topic 2:
  - 삼양홀딩스
  - 가온전선
  - 한화손해보험
  - CS홀딩스
  - 하이트진로

Main 5 companies for Topic 3:
  - 유한양행
  - 동아쏘시오홀딩스
  - 한화손해보험
  - 경방
  - 천일고속
         Topic 1   Topic 2   Topic 3   Topic 4   Topic 5
0       0.061967 -0.100643  0.362953  0.035323 -0.106740
1       0.092889  0.041630 -0.091954 -0.126238  0.041443
2       0.021629